Our objective is to deliver percentage completion rates for tasks to show progress towards our implementation goals.
First, we need to create a dataframe from an excel file which we obtain by exporting our implementation tasks from Wrike.
After importing our datafile we select only the Folder, Parent Task and Status columns.

In [96]:
import pandas as pd
import seaborn as sns
%matplotlib inline
file = 'Tenneco.xls'
df = pd.read_excel(file)
newdf = df.ix[:,['Folder','Parent task','Status']]
newdf.head()

,Folder,Parent task,Status
0,/,NaN,NaN
1,/Angola/,NaN,Completed
2,/Angola/,Initial Training,Completed
3,/Angola/,Initial Training,Completed
4,/Angola/,Initial Training,Completed


We clean the Folder column and strip out the excess "/" characters bookending the string. We leave any within the string to designate between Implementation and Trial tasks.

In [99]:
newdf['Folder'] = newdf['Folder'].map(lambda x: x.lstrip('/').rstrip('/'))
newdf = newdf.dropna()
newdf.head()

,Folder,Parent task,Status
2,Angola,Initial Training,Completed
3,Angola,Initial Training,Completed
4,Angola,Initial Training,Completed
5,Angola,Initial Training,Completed
6,Angola,Initial Training,Completed


We get a total number of tasks by counting the number of items grouped by Folder. We also get totals for each status by aggregating the tasks grouped by folder. We then create a new table and join our totals to the aggregate data.

In [111]:
aggregations = {
    'Status':{
        'Status Count': 'count',
    },
}
def newtest(row):
    if row['Status'] == 'Completed':
        return 'True'
    else:
        return 'False'
toCalc = newdf.apply(newtest,axis=1)
totals = newdf.groupby(['Folder']).count()
grouped = newdf.groupby(['Folder', 'Status']).agg(aggregations)
#grouped = grouped.drop(grouped.index[0],axis=0)
#grouped[totals.columns] = totals
#new = grouped.join(totals)
grouped

Status
                       Status Count
Folder       Status                
Angola       Active               3
             Cancelled            1
             Completed           14
Celaya       Active               2
             Cancelled            1
             Completed           16
Harrisonburg Active               1
             Cancelled            1
             Completed           18
Hartwell     Cancelled            1
             Completed           16
Kettering    Cancelled            1
             Completed           16
Napoleon     Active              10
             Completed           14
Owen Sound   Active               6
             Completed           17
Paragould    Cancelled            1
             Completed           20
Reynosa      Active               5
             Cancelled            1
             Completed           15

The join operation adds in an excess column so we drop it to clean the table.

In [72]:
new = new.drop(new.columns[2],axis=1)
#sns.swarmplot(x='(Status, Status Count)', y= 'Parent task', data=new)

In [73]:
list(new.columns.values)

[('Status', 'Status Count'), 'Parent task']

We rename the aggregate totals for status to clean up the header.

In [74]:
new.rename(columns={('Status', 'Status Count'):'Status Count'}, inplace=True)

We create a new column and calculate the percenatage for each status.

In [82]:
def percent(row):
    return row['Status']
        #return (new[('Status Count')]/new['Parent task']*100).round(2).astype(str) + '%'
new['Percent of Total'] = new.apply(percent, axis=1)
#new.index['Completed']((new[('Status Count')]/new['Parent task'])*100).round(2).astype(str) + '%'
new = new.drop(new.index[-1], axis=0)
new

KeyError: ('Status', 'occurred at index (Angola, Active)')

In [9]:
new.to_csv('out.csv')

In [11]:
new.dtypes

Status Count      int64
Parent task       int64
Percentages     float64
dtype: object